In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


#choosing test dataframe:
#df1 = pd.read_csv("../input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv")
#df2 = pd.read_csv("../input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv")
#df3 = pd.read_csv("../input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
df = pd.read_csv("../input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2017.csv", nrows=500000)

df.head()


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,...,BK,NaN,2004,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
3,8478629828,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,...,WH,NaN,2007,NaN,0,04,47-Double PKG-Midtown,NaN,NaN,NaN
4,7868300310,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,...,WHITE,NaN,2007,NaN,0,31 6,69-Failure to Disp Muni Recpt,NaN,NaN,NaN


***I've commented all the print()-s of the defs' in the code so that not to turn 
the outputs into unreadible datasets. I'm using print()-s to check the output quickly. I've run all of them to be sure they output the needful.
As an option to the print()-s I can suggest saving the defs' results in files, e.g. .csv or .json
using df.to_json('file.json', orient='records', lines=True).***

**Accuracy**
1. Numbers of houses are real; 
*#as house numbers can contain letters or symbols (e.g. '/' or '-'), I suppose here we can check for Nan value rows. We need to clarify the term 'real' with colleagues.*

2. "Registration state" of the plates exists.

In [2]:
def accuracy_check1():
    if df['House Number'].isnull().sum() > 0:
        print('Accuracy 1: FAIL')
        nan_house_numbers = df[df['House Number'].isnull()] #Having the term 'real' clarified we can 
                                                            #use np.isreal() function
        #print(f'Rows to review:\n {nan_house_numbers}')
    else:
        print('Accuracy 1: PASS')
def accuracy_check2():
    if df['Registration State'].isnull().sum() > 0:
        print('Accuracy 2: FAIL')
        nan_reg_state = df[df['Registration State'].isnull()]
        #print(f'Rows to review:\n {nan_reg_state}')
    else:
        print('Accuracy 2: PASS')
        
accuracy_check1()
accuracy_check2()

#creating filtered dataframe:
accuracy_dataframe = df.dropna(subset = ['House Number', 'Registration State'])
accuracy_dataframe.head()
accuracy_dataframe.count()

Accuracy 1: FAIL
Accuracy 2: PASS


Summons Number                       397723
Plate ID                             397689
Registration State                   397723
Plate Type                           397723
Issue Date                           397723
Violation Code                       397723
Vehicle Body Type                    396407
Vehicle Make                         394986
Issuing Agency                       397723
Street Code1                         397723
Street Code2                         397723
Street Code3                         397723
Vehicle Expiration Date              397723
Violation Location                   397368
Violation Precinct                   397723
Issuer Precinct                      397723
Issuer Code                          397723
Issuer Command                       397723
Issuer Squad                         397674
Violation Time                       397722
Time First Observed                   38206
Violation County                     396155
Violation In Front Of Or Opposit

**Timeliness**
1. We do not need data from 2016 - only 2017 data and later.

In [3]:
accuracy_dataframe = accuracy_dataframe.copy()

#editing the datatype so that we could compare with date format:
accuracy_dataframe['Issue Date'] = pd.to_datetime(accuracy_dataframe['Issue Date'])
time = pd.to_datetime('2017-01-01')

def timeliness_check1():
    if (accuracy_dataframe['Issue Date'] < time).any():
        print('Timeliness 1: FAIL')
        outdated_info = accuracy_dataframe[accuracy_dataframe['Issue Date'] < time]
        #print(f'Rows to review:\n {outdated_info}')
    else:
        print('Timeliness 1: PASS')
        
timeliness_check1()
        
#creating filtered dataframe:
timeliness_dataframe = accuracy_dataframe[
                            accuracy_dataframe['Issue Date'] >= time
                            ]
timeliness_dataframe.head()
timeliness_dataframe.count()

Timeliness 1: FAIL


Summons Number                       208986
Plate ID                             208971
Registration State                   208986
Plate Type                           208986
Issue Date                           208986
Violation Code                       208986
Vehicle Body Type                    208364
Vehicle Make                         207465
Issuing Agency                       208986
Street Code1                         208986
Street Code2                         208986
Street Code3                         208986
Vehicle Expiration Date              208986
Violation Location                   208824
Violation Precinct                   208986
Issuer Precinct                      208986
Issuer Code                          208986
Issuer Command                       208986
Issuer Squad                         208968
Violation Time                       208985
Time First Observed                   17013
Violation County                     208261
Violation In Front Of Or Opposit

**Consistency**
1. House where violation was captured is present;
*#doublechecking?* 

2. "Violation description" should exist

In [4]:
def consistency_check1():
    if timeliness_dataframe['House Number'].isnull().sum() > 0:
        print('Consistency 1: FAIL')
        nan_house_numbers = timeliness_dataframe[timeliness_dataframe['House Number'].isnull()]
        #print(f'Rows to review:\n {nan_house_numbers}')
    else:
        print('Consistency 1: PASS')
def consistency_check2():
    if df['Violation Description'].isnull().sum() > 0:
        print('Consistency 2: FAIL')
        nan_violation_descr = timeliness_dataframe[
            timeliness_dataframe['Violation Description'].isnull()
            ]
        #print(f'Rows to review:\n {nan_violation_descr}')
    else:
        print('Consistency 2: PASS')
        
consistency_check1()
consistency_check2()

#creating filtered dataframe
consistency_dataframe = timeliness_dataframe.dropna(subset = ['House Number', 'Violation Description'])
consistency_dataframe.head()
consistency_dataframe.count()

Consistency 1: PASS
Consistency 2: FAIL


Summons Number                       187684
Plate ID                             187669
Registration State                   187684
Plate Type                           187684
Issue Date                           187684
Violation Code                       187684
Vehicle Body Type                    187675
Vehicle Make                         187237
Issuing Agency                       187684
Street Code1                         187684
Street Code2                         187684
Street Code3                         187684
Vehicle Expiration Date              187684
Violation Location                   187683
Violation Precinct                   187684
Issuer Precinct                      187684
Issuer Code                          187684
Issuer Command                       187684
Issuer Squad                         187666
Violation Time                       187684
Time First Observed                   16395
Violation County                     187681
Violation In Front Of Or Opposit

**Uniqueness**
1. No double rows, no duplicates.

In [5]:
def uniqueness_check1():
    if consistency_dataframe.duplicated().sum() != 0:
        print('Uniqueness 1: FAIL')
        duplicate_rows = consistency_dataframe[consistency_dataframe.duplicated()]
        print(f'Rows to review:\n {duplicate_rows}')
    else:
        print('Uniqueness 1: PASS')
        
uniqueness_check1()

#creating filtered dataframe
uniqueness_dataframe = consistency_dataframe.drop_duplicates()
uniqueness_dataframe.head()
uniqueness_dataframe.count()

Uniqueness 1: PASS


Summons Number                       187684
Plate ID                             187669
Registration State                   187684
Plate Type                           187684
Issue Date                           187684
Violation Code                       187684
Vehicle Body Type                    187675
Vehicle Make                         187237
Issuing Agency                       187684
Street Code1                         187684
Street Code2                         187684
Street Code3                         187684
Vehicle Expiration Date              187684
Violation Location                   187683
Violation Precinct                   187684
Issuer Precinct                      187684
Issuer Code                          187684
Issuer Command                       187684
Issuer Squad                         187666
Violation Time                       187684
Time First Observed                   16395
Violation County                     187681
Violation In Front Of Or Opposit

**Validity**
1. Data types are as described by business, no outliners.(see dataset columns description on kaggle dataset page)
(#Numbers: Summons Number, Violation Code, Street Code1, Street Code2, Street Code3, Vehicle Expiration Date, Violation Precinct, Issuer Precinct, Issuer Code, Date First Observed, Law Section, Vehicle Year, Feet From Curb
#Date & Time: Issue Date. All else are plain text.)

In [6]:
#from pandas.util.testing import assert_frame_equal #This line is used in order to try assert_frame_equal.

# data for the expected dataframe values:
uniqueness_dataframe = uniqueness_dataframe.copy()
described_as_numeric_lst = [
                        'Summons Number',
                        'Violation Code',
                        'Street Code1',
                        'Street Code2',
                        'Street Code3',
                        'Vehicle Expiration Date',
                        'Violation Precinct',
                        'Issuer Precinct',
                        'Issuer Code',
                        'Date First Observed',
                        'Law Section',
                        'Vehicle Year',
                        'Feet From Curb']
#decribed_as_daytime_df = pd.DataFrame(uniqueness_dataframe['Issue Date']) #This line is used in order to try assert_frame_equal.
decribed_as_daytime_lst = ['Issue Date']
validity_expected = (described_as_numeric_lst + decribed_as_daytime_lst)
#decscribed_as_txt_lst = 

#print(uniqueness_dataframe.dtypes) #Shows numeric rows are int64 and float64 types, 
                                   #so we need to trat them both as numeric.

# data for the expected dataframe values:
as_is_numeric_df = uniqueness_dataframe.select_dtypes(include=['float64', 'int64'])
as_is_numeric_lst = as_is_numeric_df.columns.tolist()
as_is_datetime_df = pd.DataFrame(uniqueness_dataframe.select_dtypes(include=['datetime64[ns]']))
as_is_datetime_lst = as_is_numeric_df.columns.tolist()
validity_presented = (as_is_numeric_lst + as_is_datetime_lst)

#assert_frame_equal(decribed_as_daytime_df, as_is_datetime_df) #This method is used in order to try it. 
                                                         #No Errors in the output as we've already used .to_datetime for this column.


if set(validity_expected) == set(validity_presented):
    print('Validity 1: PASS')
else: 
    print('Validity 1: FAIL')
    print(f'Data type check of {set(validity_presented) - set(validity_expected)} is needed.')
    

Validity 1: FAIL
Data type check of {'Double Parking Violation', 'Violation Location', 'Hydrant Violation', 'Unregistered Vehicle?', 'No Standing or Stopping Violation'} is needed.


**Completeness**

Explore rows where data is filled according to the Business rules.
Business rules that out data MUST follow:
1. 'Vehicle Make' column should not be 'TOYOT'.
2. 'Vehicle Year' column should be >= 1976 and <="current" (2017) date.
3. No fully nulls columns.
4. No nulls or 'BLANKPLATE' in the Plate ID column.
5. "Registration state": only from 50 USA postal ANSI.
6. "Plate type" only PAS, COM.
7. Issue date - convert to dates. find min/max.
8. Violation description - not null.
9. No "Unregistered Vehicle".

In [7]:
# checking the original test dataset:
#1
toyot_rows = df[df['Vehicle Make'] == 'TOYOT']
def Completeness_1():
    if (df['Vehicle Make'] == 'TOYOT').any():
        print('Completeness 1: FAIL')
        #print(f'Rows to review:\n {toyot_rows}')
    else:
        print('Completeness 1: PASS')

Completeness_1()

#valid dataframe according to Business rule 1:
not_toyot_rows_df = df.drop(toyot_rows.index, axis = 0)


#2                     
def Completeness_2():
    xtra_vehicle_years = df[(df['Vehicle Year'] < 1976) & (df['Vehicle Year'] > 2017)]
    if len(xtra_vehicle_years) > 0:
        print('Completeness 2: FAIL')
        print(f'Rows to review:\n {xtra_vehicle_years}')
    else:
        print('Completeness 2: PASS')
        
Completeness_2()

#valid dataframe according to Business rule 2:
needed_vehicle_years_df = df[(df['Vehicle Year'] >= 1976) & (df['Vehicle Year'] <= 2017)]


#3
fully_null_columns = df.isna().all().to_dict()

def Completeness_3():
    if len(fully_null_columns) > 0:
        print('Completeness 3: FAIL\nRows to review:' )
        for column, fully_null in fully_null_columns.items():
            if fully_null:
                print(column)
    else:
        print('Completeness 3: PASS')
                
Completeness_3()

#valid dataframe according to Business rule 3:
no_fully_null_columns_df = df.dropna(how = 'all', axis = 1)


#4
def Completeness_4():
    plate_id_null = df[(df['Plate ID'].isnull()) | (df['Plate ID'] == 'BLANKPLATE')]
    if len(plate_id_null) > 0: 
        print('Completeness 4: FAIL')
        #print(f'Rows to review:\n {plate_id_null}')
    else:
        print('Completeness 4: PASS')
        
Completeness_4()

#valid dataframe according to Business rule 4:
plate_id_valid_df = df[(df['Plate ID'].notna()) & (df['Plate ID'] != 'BLANKPLATE')]


#5
USA_postal_ANSI = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 
                   'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 
                   'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 
                   'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 
                   'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 
                   'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 
                   'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

def Completeness_5():
    registration_state_false = df[~df['Registration State'].isin(USA_postal_ANSI)]
    if len(registration_state_false) > 0: 
        print('Completeness 5: FAIL')
        #print(f'Rows to review:\n {registration_state_false}')
    else:
        print('Completeness 5: PASS')
        
Completeness_5()

#valid dataframe according to Business rule 5:
registration_state_valid_df = df[df['Registration State'].isin(USA_postal_ANSI)]


#6
plate_types_valid = ['PAS', 'COM']
def Completeness_6():
    platetype_false = df[~df['Plate Type'].isin(plate_types_valid)]
    if len(platetype_false) > 0: 
        print('Completeness 6: FAIL')
        #print(f'Rows to review:\n {platetype_false}')
    else:
        print('Completeness 6: PASS')
        
Completeness_6()

#valid dataframe according to Business rule 6:
plate_types_valid_df = df[df['Plate Type'].isin(plate_types_valid)]


#7
df['Issue Date'] = pd.to_datetime(df['Issue Date']) 
issue_dates = np.array(df['Issue Date'])
time = pd.to_datetime('2017-12-31')

def Completeness_7():
    if (df['Issue Date'] > time).any():
        print('Completeness 7:')
        date_alert = df[df['Issue Date'] > time]
        print(f'Rows to review:\n {date_alert}')
    else:
        print('Completeness 7: PASS')
        
print(f'The earliest record issued on: {str(min(issue_dates))[0:10]}')
print(f'The latest record issued on: {str(max(issue_dates))[0:10]}')        
Completeness_7()

#valid dataframe according to Business rule 7:
df['Issue Date'] = pd.to_datetime(df['Issue Date']) 


#8
def Completeness_8():
    if df['Violation Description'].isnull().sum() > 0:
        print('Completeness 8: FAIL')
        nan_violation_descr = timeliness_dataframe[
            timeliness_dataframe['Violation Description'].isnull()
            ]
        #print(f'Rows to review:\n {nan_violation_descr}')
    else:
        print('Completeness 8: PASS')
        
Completeness_8()

#valid dataframe according to Business rule 8:
violation_descr_valid_df = timeliness_dataframe.dropna(subset = ['Violation Description'])


#9
#The df['Unregistered Vehicle?'] contains NaN or '0' values. 
#So we need to clarify with the colleagues what kind of data should be regarded as 
#'registered', 'non-registered' or 'false'. For now let's regard '0' as 'yes'.


def Completeness_9():
    if df['Unregistered Vehicle?'].isnull().sum() > 0:
        print('Completeness 9: FAIL')
        unregistered_vehicles_df = df.dropna(subset = ['Unregistered Vehicle?'])
        #print(f'Rows to review:\n {unregistered_vehicles_df}')
    else:
        print('Completeness 9: PASS')
        
Completeness_9()

#valid dataframe according to Business rule 9:
registered_vehicles_df = df[df['Unregistered Vehicle?'].isnull()]
registered_vehicles_df



Completeness 1: FAIL
Completeness 2: PASS
Completeness 3: FAIL
Rows to review:
No Standing or Stopping Violation
Hydrant Violation
Double Parking Violation
Completeness 4: FAIL
Completeness 5: FAIL
Completeness 6: FAIL
The earliest record issued on: 2016-07-01
The latest record issued on: 2017-06-29
Completeness 7: PASS
Completeness 8: FAIL
Completeness 9: FAIL


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,5092469481,GZH7067,NY,PAS,2016-07-10,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,5092451658,GZH7067,NY,PAS,2016-07-08,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
2,4006265037,FZX9232,NY,PAS,2016-08-23,5,SUBN,FORD,V,0,...,BK,NaN,2004,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
3,8478629828,66623ME,NY,COM,2017-06-14,47,REFG,MITSU,T,10610,...,WH,NaN,2007,NaN,0,04,47-Double PKG-Midtown,NaN,NaN,NaN
4,7868300310,37033JV,NY,COM,2016-11-21,69,DELV,INTER,T,10510,...,WHITE,NaN,2007,NaN,0,31 6,69-Failure to Disp Muni Recpt,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499994,7661039898,XAJS23,NJ,PAS,2016-11-11,37,4DSD,TOYOT,T,24890,...,WHITE,NaN,0,127-3064,0,19 7,37-Expired Muni Meter,NaN,NaN,NaN
499995,4631545940,28933AT,NY,COM,2017-03-10,36,VAN,FORD,V,0,...,YW,NaN,2009,NaN,0,NaN,PHTO SCHOOL ZN SPEED VIOLATION,NaN,NaN,NaN
499997,8022993050,GTD1952,NY,PAS,2016-08-09,70,2DSD,ME/BE,T,24690,...,WH,NaN,2014,NaN,0,41 3,70A-Reg. Sticker Expired (NYS),NaN,NaN,NaN
499998,7073905499,HKT2087,NY,PAS,2016-12-22,37,SUBN,CHEVR,T,31190,...,GY,NaN,2004,105-8344,0,03 7,37-Expired Muni Meter,NaN,NaN,NaN
